In [1]:
import sys
sys.path.append('..')

from SR_module import *
from keras.applications.vgg16 import VGG16
# from keras.applications.vgg19 import VGG19
# from keras.applications.inception_v3 import InceptionV3
from keras.preprocessing import image
from keras.applications.vgg16 import preprocess_input
from keras.models import model_from_json
from keras.models import Model
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.spatial.distance import cosine, cdist
from sklearn.decomposition import PCA
import os.path
import os

%matplotlib inline

/home/luo.sz/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
ROOT = os.environ.get('DATA', os.path.join(
    os.path.dirname(os.path.abspath('..')), 'data', 'mst'))
print(ROOT)

/home/luo.sz/beLLE/data/mst


In [3]:
# base_model = VGG16(weights='imagenet', include_top=False, pooling='max')
# model = base_model
multi = 1
if multi:
    net_name = 'BelleMultiNet_V2.json'
    net_weights = 'BelleMultiNet_Weights_V2.h5'
    layer = 10
else:
    net_name = 'BelleNet.json'
    net_weights = 'BelleNet_Weights.h5'
    layer = -3

with open(os.path.join(ROOT, net_name)) as f:
    strings = f.readline()
base_model = model_from_json(strings)
base_model.load_weights(os.path.join(ROOT, net_weights))
# model = Model(inputs=base_model.input, outputs=base_model.get_layer(index=-3).output)
model = Model(inputs=base_model.input, outputs=base_model.get_layer(index=layer).output)

+ vcg16, notop
+ vcg16, notop, pca=250, **GOOD**
+ inception, top, NOT GOOD
+ vcg19, notop,  NOT GOOD
+ vcg19, notop, pca=250, NOT GOOD

----
+ pooling = avg, pca=250
+ pooling = max, pca=250　√

In [4]:
# top20summer = ['BTCS4V1DDD3AH8', 'BBLBSYA4DD1BH8', 'BBLBGAH2DD2BK8', 'BBLBVS37DA1BH8',
#        'BBLBRP36DK1BK8', 'BBLBQJ33DD2BK8', 'BTCS4V1DDU1AH8', 'BBLBSN30DU1BK8',
#        'BBLBQJ32DU3BK8', 'BXYS5W1DDB4BL8', 'BBLBPFB6DU2BL8', 'BBLBLAC2DD1BL8',
#        'BBLBPT34DD2BH8', 'BBLBQL32DB1BL8', 'BBLBYCB5DK1AK8', 'BBLBSF32DD1BH8',
#        'BY1S4N1DDG1BT8', 'BY1S4P1DDD3AH8', 'BBLBSND2DK1BK8', 'BBLBPKB8DP1BL8'],
# top20spring = ['BEWS2N1DDA1AM8', 'B4PS2Y1DDU1AQ8', 'BY1S2D1DDA1AM8', 'BY1S2D1DDU1AM8',
#        'B4PS2Y1DDD3AQ8', 'BBLBRT22DA1AM8', 'BBLBAN20DU1AM8', 'BBLBAZ20DU1AM8',
#        'BGLS3A1DDU1AQ8', 'BBLBAP20DU2AM8', 'BY1S2H1DDD3AQ8', 'BGLS1Q1DDA2AM8',
#        'BBLBRX02DU2AQ8', 'BGLS1T1DDU1AQ8', 'BBL3VDM1DU1AQ8', 'BY1S2B1DDU1AQ8',
#        'BBLBDPC1DU1AM8', 'BGLS1Q1DDA1AM8', 'BY1S2U1DDK1AQ8', 'B4DS2J1DDU1AM8'],

In [13]:
size_file = 'pmt_sku_ss_%d_allstore.csv'
sku_file = 'pmt_size_ss_%d_allstore.csv'
season = "夏"
YEAR = 2017
year_range = lambda x : (x * 10000 + 101, x * 10000 + 1303)
target_params = {'sku_path': os.path.join(ROOT, size_file%YEAR), 
                 'size_path':os.path.join(ROOT, sku_file%YEAR),
 'season': season, 'date': year_range(YEAR), 'top_qty': None, 'top_num': 20, 'use_default_cols': False}
_, _, _, top_items_t = gen_df(**target_params)

In [14]:
L_YEAR = YEAR - 1
end_ = 1303 if season == '夏' else 1303
year_range = lambda x : (x * 10000 + 101, x * 10000 + end_)

target_params = {'sku_path': os.path.join(ROOT, size_file% L_YEAR), 
                 'size_path':os.path.join(ROOT, sku_file%L_YEAR),
 'season': season, 'date': year_range(L_YEAR), 'top_qty': 1000, 'top_num': None, }
_, _, _, top_items_b = gen_df(**target_params)

In [15]:
prop_name = '%dspring_m.csv' if season == "春" else '%dsummer_m.csv'
# prop_name = '%dprop_BL.csv'

properties = data_prop(os.path.join(ROOT, prop_name%(YEAR%100)), cols=False) 
properties_b = data_prop(os.path.join(ROOT, prop_name%(L_YEAR%100)), cols=False) 

# img_root = '/home/luo.sz/beLLE/data/imgs/'
img_root = '/home/luo.sz/beLLE/data/ygimgs/'
base_list = os.listdir(img_root)

file_list = ['%s%s.jpg'%(img_root, f) for f in properties.index.intersection(top_items_t) if '%s.jpg'%f in base_list]
file_list_b = ['%s%s.jpg'%(img_root, f) for f in properties_b.index.intersection(top_items_b) if '%s.jpg'%f in base_list]
print(len(file_list))
print(len(file_list_b))

20
79


In [16]:
features = gen_feature(model, file_list)
features_b = gen_feature(model, file_list_b)

In [17]:
use_pca = 0
kth = 3
## PCA to reduce the dimension
if use_pca:
    pca = PCA(n_components=50)
    pca = pca.fit_transform(features)
    pca_b = pca.transform(features_b)
    dist = cdist(pca, pca_b, 'cosine')
else:
    dist = cdist(features, features_b, 'cosine') # euclidean

min_threshold = dist.min(1) <=0.7

most_similar = dist.argpartition(list(range(1, kth+1)), 1)[:,:kth]

In [ ]:
pairs = {}
for i, (chosens, min_th) in enumerate(zip(most_similar, min_threshold)):

    print('===')
    if not min_th:
        print('No similar item found for the following')
    
    show_img(file_list[i], [file_list_b[chosen] for chosen in chosens])
    plt.show()
     
    index = os.path.splitext(os.path.basename(file_list[i]))[0]
    values = [os.path.splitext(os.path.basename(file_list_b[chosen]))[0] for chosen in chosens]
    pairs[index] = values
    print(index, values)
            
    

In [19]:
import pickle
brand = 'ST17'
out_name = f'summer_pair_{brand}.pkl' if season == '夏' else f'spring_pair_{brand}.pkl'
print(out_name)
with open(os.path.join(ROOT, out_name), 'wb') as f:
    pickle.dump(pairs, f)

summer_pair_ST17.pkl


In [37]:
import pickle
with open(os.path.join(ROOT, 'out', 'pairs_top3_BelleNet.pkl'), 'wb') as f:
    pickle.dump(pairs, f)

In [10]:
pca_2016_df = pd.DataFrame(pca_2016, index=index16, columns=['cnn_' + str(i) for i in range(pca_2016.shape[1])])
pca_2017_df = pd.DataFrame(pca_2017, index=index17, columns=['cnn_' + str(i) for i in range(pca_2017.shape[1])])

In [11]:
pca_2016_df.to_csv(os.path.join('/Users/shouzeluo/Desktop/Belle/data/', 'pca_2016_df.csv'))
pca_2017_df.to_csv(os.path.join('/Users/shouzeluo/Desktop/Belle/data/', 'pca_2017_df.csv'))

In [10]:
import os

img_root = '/Users/shouzeluo/Desktop/imgs'

def rename_img(img_root, year):
    dir_name = os.path.join(img_root, '{}png'.format(year))
    ref_file = os.path.join(img_root, '{}.csv'.format(year))
    k = np.loadtxt(ref_file, dtype=[('code', 'S14'),('val', int)],  delimiter=',', encoding='utf8')
    
    for code, i in k:
        os.rename(os.path.join(img_root, '{}png'.format(year), '2017png-{:d}.png'.format(i)), 
                 os.path.join(img_root, '{}png'.format(year), '{}.png'.format(code)))
    

In [8]:
def show_img(base, chosens):
    fig = plt.figure(figsize=(12,6))
    cols = len(chosens) + 1
    for i, f in enumerate([base]+chosens):
        img = plt.imread(f)
        ax = fig.add_subplot(1, cols, i+1)
        ax.set_xticks([]), ax.set_yticks([])
        ax.imshow(img)

def gen_feature(model, img_list): 

    imgs = []
    for img_path in img_list:
        img = image.load_img(img_path, target_size=(224, 224))
        x = image.img_to_array(img)
        x = np.expand_dims(x, axis=0)
        imgs.append(x)

    imgs = np.concatenate(imgs, 0)
    imgs = preprocess_input(imgs)
    imgs /= 255.
    
#     test_datagen = image.ImageDataGenerator( rescale=1./255)
    
    features = model.predict(imgs)
    features = features.reshape(features.shape[0], -1)
    
    return features



In [20]:
year_range = lambda x : (12, 14+(2 if x > 3 else 10))

In [22]:
year_range(0)

(12, 24)